<a href="https://colab.research.google.com/github/cuducquang/ML_Project/blob/main/task3_ML_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 11.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import gdown
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random
import hashlib
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import numpy as np
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_tuner as kt
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras_tuner import HyperParameters
from tensorflow import keras
import tensorflow as tf
from google.colab import files

In [3]:
drive.mount('/content/drive')

# Google Drive file ID (from the shared link)
file_id = "1imVvr3Ydu8CdhM-mjEJ0Wc7IiH5wKV5e"
file_name = "train_images.zip"
file_path = "/content/" + file_name

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", file_path, quiet=False)
print(f"Downloaded: {file_path}")

Mounted at /content/drive


Downloading...
From (original): https://drive.google.com/uc?id=1imVvr3Ydu8CdhM-mjEJ0Wc7IiH5wKV5e
From (redirected): https://drive.google.com/uc?id=1imVvr3Ydu8CdhM-mjEJ0Wc7IiH5wKV5e&confirm=t&uuid=cb026f1e-49db-43ae-b9d0-673fcec55254
To: /content/train_images.zip
100%|██████████| 820M/820M [00:16<00:00, 51.2MB/s]

Downloaded: /content/train_images.zip


In [4]:
!unzip -q $file_path -d /content/extracted_folder

In [5]:
src_root = '/content/extracted_folder/train_images'
dst_root = '/content/images_all'

os.makedirs(dst_root, exist_ok=True)

for label_folder in os.listdir(src_root):
    subdir = os.path.join(src_root, label_folder)
    if os.path.isdir(subdir):
        for filename in os.listdir(subdir):
            src_file = os.path.join(subdir, filename)
            dst_file = os.path.join(dst_root, filename)
            shutil.copy2(src_file, dst_file)

In [6]:
import pandas as pd
import os
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('/content/meta_train.csv')
df = df[df['age'] >= 45].copy()

df['image_path'] = df['image_id'].apply(lambda x: os.path.join('/content/images_all', x))

def map_age_to_group(age):
    if 45 <= age <= 49:
        return '45-49'
    elif 50 <= age <= 54:
        return '50-54'
    elif 55 <= age <= 59:
        return '55-59'
    elif 60 <= age <= 64:
        return '60-64'
    elif 65 <= age <= 69:
        return '65-69'
    elif 70 <= age <= 74:
        return '70-74'
    else:
        return '75+'

df['age_group'] = df['age'].apply(map_age_to_group)

le = LabelEncoder()
df['age_group_label'] = le.fit_transform(df['age_group'])

df = df.sample(frac=1.0, random_state=42).reset_index(drop=True)

In [7]:
from tensorflow.keras.applications.inception_v3 import preprocess_input

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

def load_and_preprocess(image_path, age, group_label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [299, 299])
    img = preprocess_input(img)
    return img, {
        'age_output': tf.cast(age, tf.float32),
        'group_output': tf.cast(group_label, tf.int32)
    }

def create_dataset(df, batch_size=32):
    paths = df['image_path'].values
    ages = df['age'].values
    groups = df['age_group_label'].values
    ds = tf.data.Dataset.from_tensor_slices((paths, ages, groups))
    ds = ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.shuffle(512).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = create_dataset(train_df)
val_ds = create_dataset(val_df)

In [8]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Dropout

model_part1 = load_model('/content/best_rice_age_model.h5')

for layer in model_part1.layers[:-10]:
    layer.trainable = False

x = model_part1.layers[-4].output
x = Dropout(0.4)(x)

age_output = Dense(1, name='age_output')(x)
group_output = Dense(7, activation='softmax', name='group_output')(x)

multi_model = Model(inputs=model_part1.input, outputs=[age_output, group_output])

multi_model.compile(
    optimizer=Adam(1e-3),
    loss={'age_output': 'mse', 'group_output': 'sparse_categorical_crossentropy'},
    loss_weights={'age_output': 1.0, 'group_output': 0.4},
    metrics={'age_output': ['mae'], 'group_output': ['accuracy']}
)

multi_model.fit(train_ds, validation_data=val_ds, epochs=5)

Epoch 1/5
261/261 ━━━━━━━━━━━━━━━━━━━━ 63s 150ms/step - age_output_loss: 3722.2090 - age_output_mae: 60.5782 - group_output_accuracy: 0.7110 - group_output_loss: 0.9027 - loss: 3722.6082 - val_age_output_loss: 1082.5518 - val_age_output_mae: 32.5809 - val_group_output_accuracy: 0.9313 - val_group_output_loss: 0.2343 - val_loss: 1081.2017
Epoch 2/5
261/261 ━━━━━━━━━━━━━━━━━━━━ 15s 58ms/step - age_output_loss: 493.8125 - age_output_mae: 19.4680 - group_output_accuracy: 0.8941 - group_output_loss: 0.3315 - loss: 493.9482 - val_age_output_loss: 22.1127 - val_age_output_mae: 3.5750 - val_group_output_accuracy: 0.9452 - val_group_output_loss: 0.2178 - val_loss: 21.4060
Epoch 3/5
261/261 ━━━━━━━━━━━━━━━━━━━━ 16s 59ms/step - age_output_loss: 60.9699 - age_output_mae: 6.1398 - group_output_accuracy: 0.9061 - group_output_loss: 0.2912 - loss: 61.0855 - val_age_output_loss: 21.8820 - val_age_output_mae: 3.5651 - val_group_output_accuracy: 0.9486 - val_group_output_loss: 0.1778 - val_loss: 21.5309

In [9]:
# PHASE 2: UNFREEZE & FINE-TUNE
for layer in multi_model.layers:
    layer.trainable = True

# Re-compile with smaller LR
multi_model.compile(
    optimizer=Adam(1e-4),
    loss={'age_output': 'mse', 'group_output': 'sparse_categorical_crossentropy'},
    loss_weights={'age_output': 1.0, 'group_output': 0.4},
    metrics={'age_output': ['mae'], 'group_output': ['accuracy']}
)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, verbose=1)

# Continue training
multi_model.fit(train_ds, validation_data=val_ds, epochs=200,
                callbacks=[early_stop, reduce_lr])

Epoch 1/200
261/261 ━━━━━━━━━━━━━━━━━━━━ 165s 343ms/step - age_output_loss: 68.5914 - age_output_mae: 6.4428 - group_output_accuracy: 0.6321 - group_output_loss: 1.5684 - loss: 69.2190 - val_age_output_loss: 180.5939 - val_age_output_mae: 9.5386 - val_group_output_accuracy: 0.4323 - val_group_output_loss: 3.6783 - val_loss: 184.2966 - learning_rate: 1.0000e-04
Epoch 2/200
261/261 ━━━━━━━━━━━━━━━━━━━━ 48s 183ms/step - age_output_loss: 63.5340 - age_output_mae: 6.2458 - group_output_accuracy: 0.5852 - group_output_loss: 1.3844 - loss: 64.0888 - val_age_output_loss: 40.8982 - val_age_output_mae: 4.5688 - val_group_output_accuracy: 0.5768 - val_group_output_loss: 1.8328 - val_loss: 42.1684 - learning_rate: 1.0000e-04
Epoch 3/200
261/261 ━━━━━━━━━━━━━━━━━━━━ 47s 181ms/step - age_output_loss: 56.2673 - age_output_mae: 5.8106 - group_output_accuracy: 0.6111 - group_output_loss: 1.2212 - loss: 56.7556 - val_age_output_loss: 85.7496 - val_age_output_mae: 7.2197 - val_group_output_accuracy: 0.51

In [10]:
multi_model.save("best_rice_age_model_part_2.h5")

In [12]:
# Google Drive file ID (from the shared link)
file_test_id = "1othgf5BTO_sZYXBOWykn2OkitSCna7J6"
file_test_name = "test_images.zip"
file_test_path = "/content/" + file_test_name

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_test_id}", file_test_path, quiet=False)
print(f"Downloaded: {file_test_path}")

Downloading...
From (original): https://drive.google.com/uc?id=1othgf5BTO_sZYXBOWykn2OkitSCna7J6
From (redirected): https://drive.google.com/uc?id=1othgf5BTO_sZYXBOWykn2OkitSCna7J6&confirm=t&uuid=b29e2d66-6e72-4cd8-b2d7-8d4433753f35
To: /content/test_images.zip
100%|██████████| 274M/274M [00:04<00:00, 62.0MB/s]

Downloaded: /content/test_images.zip


In [13]:
!unzip -q $file_test_path -d /content/extracted_test_folder

In [23]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

def predict_folder_to_csv(model, test_folder, output_csv_path):
    results = []

    for img_name in os.listdir(test_folder):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(test_folder, img_name)

            img = image.load_img(img_path, target_size=(299, 299))
            img_array = image.img_to_array(img) / 255.0
            img_batch = np.expand_dims(img_array, axis=0)

            # Predict age only
            pred = model.predict(img_batch, verbose=0)
            predicted_age = int(round(float(pred[0][0])))

            results.append({
                'filename': img_name,
                'predicted_age': predicted_age
            })

    results = sorted(results, key=lambda x: int(x['filename'].split('.')[0]))

    df = pd.DataFrame(results)
    df.to_csv(output_csv_path, index=False)
    print(f"Prediction results saved to {output_csv_path}")

model = load_model('/content/best_rice_age_model_part_2.h5', compile=False)

# Predict
predict_folder_to_csv(
    model=model,
    test_folder="/content/extracted_test_folder/test_images",
    output_csv_path="/content/predicted_age_results.csv"
)


<ipython-input-23-cd94fb2d5865>:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_age = int(round(float(pred[0][0])))


Prediction results saved to /content/predicted_age_results.csv
